# Draft Set Design

## Design Goals

- Players will draft fully playable legal decks - respecting influence
    - IDs may need to be modified to make this possible
- Collation of packs can help smooth the draft experience, but should not be super predictable
- “Rarity” should be included, and indicated to players in some fashion
- Cards should have some cross-archetype competition, parasitic cards should largely be avoided, or allow some cross canabalization
- Probably slightly lower power than current standard, perhaps startup level, but looking to make gameplay experiences fairly diverse

# Corp Design

I’d like to just provide people with PriRec’s - which means players will need to draft 18-20 points of agendas (maybe 22 if MoH is included). I’d like to keep the draft moving, but hopefully signposting and factions help push that. Supporting assets, glacier, FA, combo, and Tag-N-Bag is a big ask for a pool.

Corp decks typically will have 8-12 agendas. If players are given 60 cards that means 1/5 cards will be agendas. Econ cards typically compose 9-15 cards (though lots of overlap) so probably 1/5 cards need to be econ, perhaps slightly more. Ice bottoms out at 9 and can double to 18, so probably 1/5 of cards should be ice, 1/5 of cards should gameplan cards, and 1/5 should be a mix of value assets and additional ice.
8 players * 60 cards = 480 cards - might want a cardpool for ~10 players to make it more distinct from pod to pod - so 600 cards. Will want lots of duplicates. But going by those ratios:

120 Agendas (average points 2/agenda)

120 Econ cards (3+ credit value)

120 Ice

120 Build around pieces

120 flex cards (Ice/Assets/Upgrades)

Also need to balance influence and agendas -
each faction needs enough agendas available to support two players, so 40 points across 30 agendas - Neutrals reduce this need a bit

In [1]:
import pandas as pd
import requests

card_data = requests.request(url="https://netrunnerdb.com/api/2.0/public/cards",method="GET").json()['data']
cards_df = pd.DataFrame(card_data)
cards_df = cards_df.query("pack_code not in ['draft','tdc']")

trashorbusto_data = requests.request(url="https://trash-or-busto.herokuapp.com/api/rankings",method="GET").json()
trashorbusto_corp_df = pd.DataFrame(trashorbusto_data['corp_cards'])
trashorbusto_corp_df['rating'] = pd.to_numeric(trashorbusto_corp_df['rating']).round(0)


In [2]:
pd.merge(
    cards_df[cards_df['type_code'] == 'agenda'],
    trashorbusto_corp_df[trashorbusto_corp_df['rating'] > 1200],
    how='inner',
    left_on='code',
    right_on='nrdb_key'
).to_csv("CorpAgendasGreater1200.csv")


## Thoughts After Looking at Agendas

I don't want to include PE in the pool - I think it's a bit parasitic. That doesn't mean net damage won't be part of it, but I think taking it out of the pool can hopefully give net damage a slightly different look

Have 84 points of Neutral Corp Agendas on the first pass -> so would need about 30 points/faction
Generally finding ~70 points of agendas fit easily, with ~10-20 points that support specific gameplans
So will tighten things up. Need to make a decision about 3x Astro

Archtypes defined by agendas:
- Brain Rewiring Kill (would require CI/PD/Cybernetics Court)
- EffComm Combo (SfSS, DediRecoc)
- Regulatory Capture (Is this parasitic bad pub Outfit? - can you work this without the ID?)
- Obokata/Blood in the Water (Handsize matters)

# Archtype Overviews:

- NBN
    - Fast Advance (NEH/Astroscript/SSCG/SfSS/VladGrid)
    - Tagstorm (wide/tall) - Midseasons/HHN/Ice/ARES -> Psycho/TnB/SfV
    - Glacier
- HB
    - Brain Rewiring Combo
    - EffComm Combos
    - Never-advance (Jeeves/Seamless Launch)
    - Bioroid Tribal
- Jinteki
    - AgInf/RP glacier
    - Tempo/Grinder traps
    - IG Prison?/Combo (probably want to lean on the combo)
- Weyland
    - Gagarin Assets
    - Argus (try and support 2x decks here - more FA, more kill)
    - Ob Combo Lord

In [3]:
r = requests.request(method="GET",url="https://netrunnerdb.com/api/2.0/public/decklists/by_date/2023-02-28")

In [4]:
from datetime import date, timedelta
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = date(2023,2,22)
end_date = date.today()

for day in daterange(start_date=start_date, end_date=end_date):
    print(day.strftime("%Y-%m-%d"))

2023-02-22
2023-02-23
2023-02-24
2023-02-25
2023-02-26
2023-02-27


In [23]:
import tqdm
from ratelimit import limits, sleep_and_retry

@sleep_and_retry
@limits(calls=1, period=timedelta(seconds=0.25).total_seconds())
def get_decks(date_string, card_dictionary):
    r = requests.request(method="GET",url=f"https://netrunnerdb.com/api/2.0/public/decklists/by_date/{date_string}")

    for decklist in r.json()['data']:
        for cid, count in decklist['cards'].items():
            try:
                card_dictionary[cid]['count'] += count
                card_dictionary[cid]['decklists'] += 1
            except KeyError:
                card_dictionary[cid] = {'count': count, 'decklists':1}
    return card_dictionary

# card_dict = {}

# for year in tqdm.tqdm(range(2013,2022)):
#     start_date = date(year,10,1)
#     end_date = date(year,12,1)
#     for formatted_date in daterange(start_date=start_date, end_date=end_date):
#         get_decks(card_dictionary=card_dict, date_string=formatted_date)


100%|██████████| 9/9 [06:39<00:00, 44.36s/it]


In [26]:
# worldsish_annual_data = pd.DataFrame.from_dict(card_dict,orient="index")
# corp_data = pd.merge(
#     cards_df[cards_df['type_code'] == 'agenda'],
#     trashorbusto_corp_df[trashorbusto_corp_df['rating'] > 1200],
#     how='inner',
#     left_on='code',
#     right_on='nrdb_key'
# )
# corp_data = pd.merge(
#     corp_data,
#     worldsish_annual_data,
#     left_on='code',
#     right_index=True
# )
# corp_data.to_csv("CorpCardsDecklistFrequency.csv")
corp_data = pd.read_csv("CorpCardsDecklistFrequency.csv")
corp_data['copies_per_deck'] = corp_data['count']/corp_data['decklists']
corp_data[['faction_code','title','rating','count','decklists','copies_per_deck']]

,faction_code,title,rating,count,decklists,copies_per_deck
0,haas-bioroid,Accelerated Beta Test,1841.0,1236,441,2.802721
1,nbn,AstroScript Pilot Program,2132.0,1127,484,2.328512
2,nbn,Breaking News,2055.0,872,378,2.306878
3,weyland-consortium,Posted Bounty,1423.0,243,121,2.008264
4,haas-bioroid,Mandatory Upgrades,1470.0,208,96,2.166667
...,...,...,...,...,...,...
129,nbn,License Acquisition,1431.0,34,13,2.615385
130,nbn,Project Beale,1870.0,154,57,2.701754
131,weyland-consortium,Hostile Takeover,1905.0,153,60,2.550000
132,weyland-consortium,Oaktown Renovation,1708.0,55,22,2.500000


In [35]:
corp_data = pd.merge(
    cards_df,
    trashorbusto_corp_df[trashorbusto_corp_df['rating'] > 1200],
    how='inner',
    left_on='code',
    right_on='nrdb_key'
)
corp_data = pd.merge(
    corp_data,
    worldsish_annual_data,
    left_on='code',
    right_index=True
)

corp_data.to_csv("corp_data_torb_and_counts.csv")
corp_data[['faction','title','ranking','count','decklists','cost','strength','keywords']]

,faction,title,ranking,count,decklists,cost,strength,keywords
0,haas-bioroid,Haas-Bioroid: Engineering the Future,12,229,229,NaN,NaN,Megacorp
1,haas-bioroid,Accelerated Beta Test,75,1236,441,NaN,NaN,Research
2,haas-bioroid,Experiential Data,882,67,36,2.0,NaN,NaN
3,jinteki,Precognition,815,90,43,0.0,NaN,NaN
4,jinteki,Chum,666,405,198,1.0,4.0,Code Gate
...,...,...,...,...,...,...,...,...
816,weyland-consortium,Punitive Counterstrike,93,85,35,3.0,NaN,Black Ops
817,weyland-consortium,Crisium Grid,90,74,52,3.0,NaN,Region
818,neutral-corp,PAD Campaign,281,109,47,2.0,NaN,Advertisement
819,neutral-corp,Enigma,135,122,66,3.0,2.0,Code Gate
